In [1]:
import math

In [6]:
class SlabDesign:
    def __init__(self, L, S, dead, live, Ln, Sn, fc, fy, supp_type, wc, Cc, d_main, d_temp, spans, restrain, sw, ll_red):
        self.L = L  # mm
        self.S = S  # mm
        self.dead = dead  # KPa
        self.live = self.live_load_reduction(L, S, live, ll_red)
        self.slab_type = self.condition(L, S)
        self.Ln = Ln
        self.Sn = Sn
        self.fc = fc
        self.fy = fy
        self.t = self.minimum_thickness(supp_type, fc, fy, S, wc)
        self.sw = self.self_weight(self.t, sw)
        self.Cc = Cc
        self.d_main = d_main
        self.d_temp = d_temp
        self.d = self.effective_depth(self.t, Cc, d_main)
        self.w_ult = self.ultimate(self.dead, self.live, self.sw)
        self.aci = self.ACI_coefficient(spans, restrain)
        self.Mu = self.Moments(self.aci, Sn, self.w_ult)
        self.r_ratio = self.reinforcement_ratio(self.Mu, self.d, fc, fy)
        self.p_temp = self.reinforcement_ratio_temp(fy)
        self.b1 = self.BetaOne(fc)
        self.p_max = self.rho_max(fc, fy, self.b1)
        self.p_min = self.rho_min(fc, fy)
        self.As_main = self.steel_area(self.r_ratio, self.p_max, self.p_min, self.p_temp, self.d)
        self.As_temp = self.temperature_bars(self.p_temp, self.d)
        self.A_bar = self.Bar_area(d_main)
        self.A_temp = self.Temp_area(d_temp)
        self.main_s = self.main_bar_spacing(self.A_bar, self.As_main, self.t)
        self.temp_s = self.temp_spacing(self.A_temp, self.As_temp, self.t)

    def condition(self, Long, Short):
        return 'TwoWay' if Long / Short <= 2 else 'OneWay'

    def live_load_reduction(self, Long, Short, live, ll_red):
        if ll_red == 'Yes' and Long * Short > 15 * 1000**2:
            R = 0.08 * (Long * Short * (1 / 1000)**2 - 15)
            return live * (1 - R)
        return live

    def minimum_thickness(self, supp_type, fc, fy, S, wc):
        supp_types = {
            'Simply Supported': S / 20,
            'One End': S / 24,
            'Both Ends': S / 28,
            'Cantilever': S / 10
        }
        t = supp_types.get(supp_type, 'Invalid')

        if t == 'Invalid':
            raise ValueError('Invalid Input. Choose From (Simply Supported, One End, Both Ends, Cantilever)')

        if 1440 <= wc <= 1840:
            t *= max(1.65 - 0.0003 * wc, 1.09)

        if fy < 414:
            t *= (0.4 + (fy / 700))

        return max(100, min(math.ceil(t / 25) * 25, 250))

    def self_weight(self, t, sw):
        return sw * (t / 1000)

    def effective_depth(self, t, Cc, d_main):
        return t - Cc - d_main / 2

    def ultimate(self, dead, live, sw):
        return max(1.2 * (dead + sw) + 1.6 * live, 1.4 * (dead + sw))

    def ACI_coefficient(self, spans, restrain):
        coefficients = {
            'Unrestrained': [0, 1/11, 1/10, 1/11, 1/16, 1/11],
            'Spandrel': [1/24, 1/14, 1/10, 1/11, 1/16, 1/11],
            'Column': [1/16, 1/14, 1/10, 1/11, 1/16, 1/11]
        }
        if spans > 2:
            return coefficients[restrain]
        return coefficients[restrain][:3]

    def Moments(self, aci, Sn, wu):
        return [i * wu * Sn**2 for i in aci]

    def reinforcement_ratio(self, M, d, fc, fy):
        R = [(m) / (0.9 * 1000 * d**2) for m in M]
        return [((0.85 * fc) / fy) * (1 - math.sqrt(1 - (2 * i) / (0.85 * fc))) for i in R]

    def reinforcement_ratio_temp(self, fy):
        if fy < 420:
            return 0.002
        return max((0.0018 * 420) / fy, 0.0014)

    def BetaOne(self, fc):
        if 17 <= fc <= 28:
            return 0.85
        if 28 < fc < 55:
            return 0.85 - 0.05 * (fc - 28) / 7
        if fc >= 55:
            return 0.65
        raise ValueError("fc must be greater than or equal to 17 MPa")

    def rho_max(self, fc, fy, beta):
        return (3 / 8) * ((0.85 * fc * beta) / fy)

    def rho_min(self, fc, fy):
        return max(math.sqrt(fc) / (4 * fy), 1.4 / fy)

    def steel_area(self, r_ratio, p_max, p_min, p_temp, d):
        Area_steel = []
        for ratio in r_ratio:
            if p_min < ratio < p_max:
                As = max(ratio, p_temp) * 1000 * d
            elif ratio >= p_max:
                raise ValueError('Increase Slab Thickness')
            else:
                As = max(min((4 / 3) * ratio, p_min), p_temp) * 1000 * d
            Area_steel.append(As)
        return Area_steel

    def temperature_bars(self, p_temp, d):
        return p_temp * 1000 * d

    def Bar_area(self, ds):
        return (math.pi / 4) * ds**2

    def Temp_area(self, dtemp):
        return (math.pi / 4) * dtemp**2

    def main_bar_spacing(self, A_bar, As_main, t):
        main_spacing = [min((1000 * A_bar) / m, 3 * t, 450) for m in As_main]
        return [math.floor(s / 25) * 25 for s in main_spacing]

    def temp_spacing(self, A_temp, As_temp, t):
        s_temp = min((1000 * A_temp) / As_temp, 5 * t, 450)
        return math.floor(s_temp / 25) * 25
#--------------------------------Inputs---------------------------------
# Inputs
if __name__ == "__main__":
    # Define inputs
    L = 7000  # mm
    S = 3250  # mm
    dead_load = 2.6  # KPa
    live_load = 2.4  # KPa
    Ln = 6750  # mm
    Sn = 3000  # mm
    fc = 28  # MPa
    fy = 275  # MPa
    supp_type = 'Both Ends'
    wc = 2400  # kg/m3
    Cc = 20  # mm
    d_main = 12  # mm
    d_temp = 10  # mm
    spans = 3
    restrain = 'Spandrel'
    sw = 23.544  # kN/m^3
    ll_red = "No"
    
    # Create SlabDesign object
    slab = SlabDesign(L, S, dead_load, live_load, Ln, Sn, fc, fy, supp_type, wc, Cc, d_main, d_temp, spans, restrain, sw, ll_red)

    # Print results
    print("Slab Type:", slab.slab_type)
    print("Effective Depth (d):", slab.d)
    print("Ultimate Load (w_ult):", slab.w_ult)
    print("Moments (Mu):", slab.Mu)
    print("Reinforcement Ratio:", slab.r_ratio)
    print("Main Steel Area (As_main):", slab.As_main)
    print("Temperature Steel Area (As_temp):", slab.As_temp)
    print("Main Bar Spacing:", slab.main_s)
    print("Temperature Bar Spacing:", slab.temp_s)
    print(f'Live load reduction, {ll_red}: Use LL={slab.live} kPa')


Slab Type: OneWay
Effective Depth (d): 74.0
Ultimate Load (w_ult): 9.78528
Moments (Mu): [3669479.9999999995, 6290537.142857143, 8806752.0, 8006138.181818182, 5504220.0, 8006138.181818182]
Reinforcement Ratio: [0.0027512133742666647, 0.004773017930990591, 0.006762137587450098, 0.006123899118220002, 0.0041612665665521475, 0.006123899118220002]
Main Steel Area (As_main): [271.45305292764425, 376.72727272727263, 500.3981814713072, 453.16853474828014, 376.72727272727263, 453.16853474828014]
Temperature Steel Area (As_temp): 148.0
Main Bar Spacing: [300, 300, 225, 225, 300, 225]
Temperature Bar Spacing: 450
Live load reduction, No: Use LL=2.4 kPa
